In [46]:
import os
import shutil
import zipfile
import pandas as pd
import time
import datetime
import subprocess

In [47]:
# output directory to store the results
outDir = "/Users/duanyueyun/Desktop/Capstone/Decompiled_files"
# input directory that contains the APK files
inDir = "/Users/duanyueyun/Desktop/Capstone/APK_files"
# directory to d2j-dex2jar.sh
dex2jar = "/Users/duanyueyun/Desktop/Capstone/Decompile_tools/dex2jar-2.0/d2j-dex2jar.sh"
# directory to jd-cli.jar
jd_cli = "/Users/duanyueyun/Desktop/Capstone/Decompile_tools/jd-cli-1.1.0.Final-dist/jd-cli.jar"
# directory of this notebook
notebook = "/Users/duanyueyun/Desktop/Capstone/"
# directory to store error files
errorDir = "/Users/duanyueyun/Desktop/Capstone/error_files"

In [48]:
# get all apk filenames
apks=os.listdir(inDir)

In [49]:
# just check all files have been downloaded, can skip
repack_pairs = pd.read_csv("~/Desktop/Capstone/Download apk/repackaging_pairs.txt", sep=',')
mine = repack_pairs.iloc[12001:,:]
sha256s = list(mine['SHA256_ORIGINAL'].unique())
sha256s += list(mine['SHA256_REPACKAGE'].unique())
downloaded = [apk.split('.')[0] for apk in apks]
for i in sha256s:
    if i not in downloaded:
        print(i)

In [50]:
start = 1000
end = 1200
errors = []
for i in range(start, end):

    apk = apks[i]
    apk_name=apk.split('.')[0]
    
    # skip files with extension other than apk
    if apk.split('.')[1]!='apk':
        continue
    
    # create a path named by the app's SHA256
    output_folder = os.path.join(outDir, apk_name)
    
    # skip if the app has already been processed
    if os.path.exists(output_folder):
        continue
    
    print(datetime.datetime.now().time(), i)
    # create the folder
    os.mkdir(output_folder)

    # path of the .jar file
    output_jar = os.path.join(output_folder, apk_name+'.jar')
    # path of the apk file
    in_apk = os.path.join(inDir, apk)
    # obtain the .jar file
    os.system("sh {} -f -o {} {}".format(dex2jar, output_jar, in_apk))
    
    # create a folder named java_sources to store the java files
    jar_folder = os.path.join(output_folder, "java_sources")
    os.mkdir(jar_folder)
    
    # obtain the java files
    cmd = "java -jar {} -od {} {}".format(jd_cli, jar_folder, output_jar)
    
    try:
        subprocess.run(cmd,timeout=60,shell=True)
        
    except subprocess.TimeoutExpired:
        if os.path.exists(os.path.join(notebook, 
                                       apk_name+'-error.zip')):
            errors.append(apk_name)
            shutil.move(os.path.join(notebook, apk_name+'-error.zip'), 
                        os.path.join(errorDir, apk_name+'-error.zip'))

    if os.path.exists(os.path.join(notebook, 
                                       apk_name+'-error.zip')):
        errors.append(apk_name)
        shutil.move(os.path.join(notebook, apk_name+'-error.zip'), 
                    os.path.join(errorDir, apk_name+'-error.zip'))            
            
    # remove the .jar file
    os.remove(output_jar)
    
    # obtain xml file
    other_folder = os.path.join(output_folder, "other")
    os.system('apktool d {} -o {}'.format(in_apk, other_folder))
    
    # delete files that we do not want, keep only .xml
    for file in os.listdir(other_folder):
        if file == 'AndroidManifest.xml':
            continue
        path = os.path.join(other_folder, file)
        
        if os.path.isdir(path):
            shutil.rmtree(path)
        else:
            os.remove(path)
            

10:30:11.204187 1000
10:30:22.822640 1001
10:30:30.078775 1002
10:30:40.255269 1003
10:32:50.104839 1004
10:33:12.985106 1005
10:33:54.171493 1006
10:34:10.982586 1007
10:34:42.120788 1008
10:35:05.562334 1009
10:35:15.620545 1010
10:35:58.325918 1011
10:36:08.936896 1012
10:36:16.935786 1013
10:36:24.849972 1014
10:36:41.756059 1015
10:37:24.625953 1016
10:37:34.837939 1017
10:37:49.018435 1018
10:38:06.273466 1019
10:40:14.121942 1020
10:40:57.326572 1021
10:41:03.963650 1022
10:43:13.647895 1023
10:43:27.420391 1024
10:45:39.841497 1025
10:46:37.191521 1026
10:47:01.499061 1027
10:47:21.532343 1028
10:47:30.019363 1029
10:48:07.821338 1030
10:48:51.847948 1031
10:49:20.675692 1032
10:49:36.194612 1033
10:49:51.354999 1034
10:50:13.760247 1035
10:50:59.289634 1036
10:51:14.911876 1037
10:51:31.367584 1038
10:51:46.231673 1039
10:51:57.027116 1040
10:52:42.847665 1041
10:52:56.303322 1042
10:53:10.723690 1043
10:53:43.667289 1044
10:54:02.429423 1045
10:54:49.051787 1046
10:55:00.5196

In [51]:
errors

['997A5E20E2D08B391B21E062652A9AF612FD9E2E625D4A62A54B680AB9C23A1E',
 '79B54C8CFB0785507300CE439D8A62BA55AC3688F2A518ECBD1F1AF0D9C634C5',
 'F0E1D359245EFAAFE0A901681B7394421049BA440A172A560E41F5A14FB587B2',
 '2608AB47D145ACC619F5CDAC674A30A509D887D90F59FD48924EB7659131BAA0',
 'EBA0129A22A618BB049950600F5FEEDADD990ED147C42967007B480DAA7B0F67',
 '5BFB9089527BBCD74EF53C356DBD02258B24CA1CB2C0BEE491797B71AD402CA4']

In [52]:
if os.path.exists('/Users/duanyueyun/Desktop/Capstone/error_list.txt'):
    mode = 'a'
else:
    mode = 'w'

error_file = open('/Users/duanyueyun/Desktop/Capstone/error_list.txt', mode)
        
for apk in errors:

    error_file.write("%s," % apk)

error_file.close()